TODO:
1) compute expected return based on market risk and estimated risk (fill price based)
2) BUG: risk	market_risk in final resutls seem to be in wrong order implyting something is not being calculated correctly.  must debug this.

In [1]:
# Install yfinance if not already installed
#!pip install yfinance

from IPython.display import display

from tqdm import tqdm 
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def get_historical_returns(ticker, interval=5):
    # Fetch historical data for the stock ticker
    stock_data = yf.download(ticker, start="2000-01-01", end="2024-12-31")
    
    # Calculate the percentage change for each day
    stock_data['Return'] = stock_data['Adj Close'].pct_change()
    
    # Calculate returns over X-day intervals
    stock_data[f'{interval}-Day Return'] = stock_data['Adj Close'].pct_change(periods=interval)
    
    # Drop NaN values that arise from pct_change()
    stock_data.dropna(inplace=True)
    
    return stock_data


def days_until(date_str):
    # Convert the given date string to a datetime object
    target_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Get the current date
    current_date = datetime.today()
    
    # Calculate the difference in days
    difference_in_days = (target_date - current_date).days
    
    return difference_in_days


def get_option_chains_within_45_days(ticker):
    """
    Get the option chains for all expiration dates within the next 45 days for the given ticker.
    """
    # Get the stock ticker object
    stock = yf.Ticker(ticker)
    
    # Get the available expiration dates
    expirations = stock.options
    
    # Calculate the cutoff date for 45 days from now
    cutoff_date = datetime.now() + timedelta(days=45)
    
    # Filter expiration dates within the next 45 days
    filtered_expirations = [exp for exp in expirations if datetime.strptime(exp, '%Y-%m-%d') <= cutoff_date]
    
    if not filtered_expirations:
        #print(f"No expiration dates within the next 45 days for {ticker}.")
        return None
    
    #print(f"Expiration dates within the next 45 days for {ticker}: {filtered_expirations}")
    
    # Get the option chains for all filtered expiration dates
    option_chains = {}
    for expiration in filtered_expirations:
        option_chain = stock.option_chain(expiration)
        option_chains[expiration] = option_chain
    
    return option_chains


def print_option_chain(option_chain, ticker, expiration_date):
    """
    Print the option chain for the given ticker and expiration date.
    """
    print(f"\nOption Chain for {ticker} expiring on {expiration_date}\n")
    print("Calls:")
    display(option_chain.calls.head())
    print("\nPuts:")
    display(option_chain.puts.head())


def get_current_stock_price(ticker):
    """
    Get the current stock price for the given ticker.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period="3mo")
    current_price = history["Close"].iloc[-1]
    sma_10 = history["Close"].tail(10).mean()
    sma_50 = history["Close"].tail(50).mean()
    return current_price, sma_10, sma_50


def dict_to_df(data):
    """
    Convert a dictionary to a pandas DataFrame.
    Ensures that the output is always a DataFrame.
    
    Parameters:
    data (dict): Dictionary to convert.
                 If the dictionary represents a single row, the function will
                 convert it to a DataFrame with one row.
    
    Returns:
    pd.DataFrame: DataFrame representation of the dictionary.
    """
    if isinstance(data, dict):
        # If the dictionary represents a single row, wrap it in a list to ensure one row DataFrame
        df = pd.DataFrame([data]).T
    else:
        # If the data is already in a format suitable for a DataFrame, just convert it
        df = pd.DataFrame(data)
    
    return df


def find_put_spread(puts_f, last_f, margin=.1):

    short_put_candidates = puts_f[puts_f['strike'] <= last_f]
    put_spreads = []
    for _, short_put in short_put_candidates.iterrows():
        long_put_candidates = puts_f[puts_f['strike'] < short_put['strike']]
        if not long_put_candidates.empty:
            for _, long_put in long_put_candidates.iterrows():
            #long_put = long_put_candidates.iloc[0]  # Get the first available long put
                # Calculate credit and risk
                short_price = (short_put['bid']*.5 + short_put['ask']*.5)
                long_price = (long_put['bid']*.5+long_put['ask']*.5)
                credit = short_price - long_price#short_put['bid'] - long_put['ask']
                market_credit = short_put['bid']-long_put['ask']
                risk = short_put['strike'] - long_put['strike'] - credit
                market_risk = short_put['strike'] - long_put['strike'] - market_credit

                # Check if credit is greater than 50% of the risk   
                if credit > margin * risk:
                    put_spreads.append({
                        'short_put_contract': short_put['contractSymbol'],
                        'short_put_strike': short_put['strike'],
                        'short_bid':short_put['bid'],
                        'short_ask':short_put['ask'],
                        'short_price':short_price,
                        'long_put_contract': long_put['contractSymbol'],
                        'long_put_strike': long_put['strike'],
                        'long_bid':long_put['bid'],
                        'long_ask':long_put['ask'],
                        'long_price':long_price,
                        'last_price':last_f,
                        'credit': credit,
                        'market_credit':market_credit,
                        'risk': risk,
                        'market_risk':market_risk,
                        'c/r': credit/risk,
                        'market_c/r':market_credit/market_risk,
                        'ITM_return':(short_put['strike']-last_f)/last_f, 
                        'expected_return':''             
                    })

    return pd.DataFrame(put_spreads)

In [3]:
# Get option chains and current stock prices for Google and Apple
MARGIN = 0.1
tickers = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "FB", "TSLA", "BRK.B", "NVDA", "JPM",
    "JNJ", "V", "UNH", "PG", "HD", "BAC", "DIS", "MA", "PYPL", "VZ","MSTR",
    "ADBE", "NFLX", "INTC", "CMCSA", "PFE", "KO", "PEP", "T", "XOM", "CSCO",
    "ABT", "MRK", "NKE", "CRM", "LLY", "TMO", "AVGO", "WMT", "ACN", "COST",
    "DHR", "NEE", "MCD", "MDT", "WFC", "HON", "UNP", "LIN", "TXN", "AMGN",
    "MS", "PM", "BMY", "QCOM", "SCHW", "C", "IBM", "ORCL", "BA", "UPS",
    "GS", "SBUX", "RTX", "BLK", "CAT", "CVX", "LOW", "SPGI", "AMT", "INTU",
    "MMM", "ISRG", "MDLZ", "GE", "AXP", "PLD", "ADP", "CB", "MO", "ZTS",
    "CCI", "SYK", "TGT", "DE", "BDX", "SCHW", "MMC", "DUK", "CI", "PGR",
    "SO", "USB", "TFC", "EL", "ADI", "ITW", "HUM", "AMAT", "NOC", "MU",
    "BKNG", "GM", "GILD", "FIS", "LMT", "TJX", "NSC", "FISV", "CSX", "HCA",
    "PNC", "ICE", "COP", "MET", "EW", "VRTX", "TRV", "ETN", "CME", "PSA",
    "MCO", "GD", "MPC", "ADSK", "MAR", "KMB", "AON", "FDX", "ADP", "ECL",
    "NOC", "APH", "JCI", "DG", "COF", "SPG", "KLAC", "MNST", "AZO", "APTV",
    "FTNT", "EOG", "ROST", "BAX", "RMD", "CTAS", "KMI", "O", "WMB", "IQV",
    "MCK", "BSX", "D", "CDNS", "AEP", "SHW", "TT", "EA", "PEG", "RSG",
    "STZ", "ATO", "EXC", "CTSH", "PPG", "WY", "DOV", "PSX", "ALL", "WEC",
    "SRE", "CMG", "PCAR", "HSY", "CARR", "ILMN", "AME", "AVB", "SLB", "BKR",
    "ZBH", "MSCI", "PH", "HPQ", "HES", "DTE", "ES", "DLR", "DVN", "OKE",
    "FAST", "AFL", "VFC", "CERN", "MSI", "DHI", "EBAY", "ED", "MTB", "SYY",
    "WAT", "AIG", "RCL", "FMC", "MKC", "LYB", "FTV", "EMN", "EMR", "CMS",
    "CSL", "ATO", "KEYS", "BBY", "HRL", "FLT", "TSCO", "GWW", "XRAY", "LUV",
    "FFIV", "TDY", "XEL", "REGN", "WDC", "TROW", "WYNN", "HIG", "J", "DISH",
    "CAG", "UAL", "A", "AKAM", "PWR", "COO", "MAS", "NRG", "BBY", "ETR",
    "ANET", "PAYC", "DD", "CTXS", "DRE", "SEE", "ZBRA", "ESS", "ODFL", "DPZ",
    "SBAC", "AEE", "ABC", "LNT", "NTAP", "K", "MKTX", "CINF", "MRO", "IR",
    "VRSK", "NDAQ", "HST", "ROK", "ALLE", "KMX", "LH", "EXR", "ANSS", "LDOS",
    "TDG", "TECH", "NRG", "NTAP", "CNP", "HII", "AAP", "WRB", "ALGN", "CBOE",
    "HOLX", "AES", "LHX", "CPT", "CCL", "CMS", "MAS", "MTD", "NLOK", "EXPD",
    "HAS", "GPC", "IP", "OKE", "VMC", "IPG", "PKI", "ETSY", "WST", "AVY",
    "HWM", "XRAY", "CFG", "BEN", "CDW", "NTRS", "LUMN", "PXD", "POOL", "PEAK",
    "RF", "UDR", "CMA", "DRI", "FRT", "AES", "VTRS", "WHR", "PBCT", "JPM",
    "FTI", "TAP", "WAB", "CLX", "FE", "AES", "CMS", "AWK", "PPL", "PNW",
    "ESS", "FITB", "BF.B", "OKE", "DLR", "ATO", "ESS", "CMS", "VTRS", "TAP",
    "EXPE", "ABMD", "ARE", "MAA", "UDR", "FMC", "BRX", "MPWR", "WYNN", "FRT",
    "PPL", "ATO", "CMS", "VTRS", "CPT", "UDR", "HRB", "ZION", "BEN", "WY",
    "HBAN", "PFG", "MTB", "CFG", "GPC", "EVRG", "SLG", "SBAC", "STT", "FRC"
]


In [ ]:
put_spreads_df = []
for ticker in tqdm(tickers):
    try:
        option_chains = get_option_chains_within_45_days(ticker)
        exp_dates = list(option_chains.keys())
        for e_d in exp_dates:
            # stock return distribution and compute expected returns
            days = days_until(date_str=e_d)+1  # this should be moved into find put spread function
            if days < 1: continue
            return_dist = get_historical_returns(ticker, interval=days) # this should be moved into find put spread function

            # put trade
            puts = option_chains[e_d].puts
            puts = puts[puts.inTheMoney==False]
            last, sma_10, sma_50 = get_current_stock_price(ticker)
            puts=puts.sort_values(by='strike', ascending=False)
            
            put_spread = find_put_spread(puts_f=puts, last_f=last, margin=MARGIN)

            if not put_spread.empty: # this should be moved into find put spread function
                
                vals = return_dist[f'{days}-Day Return'].values
                p = np.zeros(put_spread.shape[0])
                for idx, itm_val in enumerate(put_spread['ITM_return']):
                    p[idx] = 1-len(vals[vals<itm_val])/len(vals)

                return_break_even = ((put_spread['short_put_strike']-put_spread['credit']) - put_spread['last_price'])/put_spread['last_price']
                p_break_even = np.zeros(put_spread.shape[0])
                for idx, bke_val in enumerate(return_break_even):
                    p_break_even[idx] = 1-len(vals[vals<bke_val])/len(vals)
                    
                put_spread['P (success)']=p
                put_spread['P(gain)']=p_break_even
                put_spread['P(loss)']=1-p_break_even
                put_spread['break_even_return'] = return_break_even
                put_spread["expected_value"] = p*put_spread['credit'] + (p_break_even-p)*put_spread['credit']/2 - (1-p_break_even)*put_spread['risk']
                put_spread["expected_return"] = (put_spread["expected_value"])/put_spread['risk']
                put_spread["expected_annual_return"] = (put_spread["expected_return"])/(days/365)

                put_spread["expected_market_value"] = p*put_spread['market_credit'] - (1-p)*put_spread['market_risk']
                put_spread["expected_market_return"] = (put_spread["expected_market_value"])/put_spread['market_risk']
                put_spread["expected_annual_market_return"] = (put_spread["expected_market_return"])/(days/365)

                put_spread['days']=days
                put_spread['sma_10']=sma_10
                put_spread['sma_50']=sma_50
                put_spreads_df.append(put_spread)
    except:  continue    
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
return_break_even
put_spread
#1-len(vals[vals<put_spread['ITM_return']])/len(vals)

In [ ]:
all_apreads=pd.concat(put_spreads_df)

final=pd.concat(put_spreads_df)
gap = (final.short_put_strike-final.last_price)/final.last_price
final['gap']=gap

final['Credit/SmL'] = final.credit/(final.short_put_strike - final.long_put_strike)
final['date']=datetime.today().date()

#select1 = final['ITM_return']>-0.2
#select2 = (final['long_ask'] - final['long_bid'])/final['long_ask']<0.4
#select3 = (final['short_ask'] - final['short_bid'])/final['short_ask']<0.4
select4 = final['expected_annual_return']>0
#final = final[select1 & select2 & select3 & select4]
final = final[select4]

final = final.sort_values(by='expected_annual_return', ascending=False) 


In [ ]:
final.to_csv('../data/put_spread_trades.csv', index=False)

In [ ]:
df = pd.read_csv('../data/put_spread_trades_all.csv')
updated_df = pd.concat([df, final], ignore_index=True)
updated_df.to_csv('../data/put_spread_trades_all.csv', index=False)

In [ ]:
pd.set_option('display.max_columns', None)

final.head()

In [ ]:
def plot_function(df, xcol, ycol,title):
    try:
        fig = px.scatter(df, x=xcol, y=ycol, 
            hover_data=[
                'short_put_contract', 
                'long_put_contract', 
                'days', 
                'ITM_return',
                'c/r',
                'P(success)',
                'expected_annual_market_return',
                'last_price',
                'sma_10','sma_50'],)
        fig.update_layout(title=title)
        fig.show()
    except: print("Plotting failed")

temp = final[final['expected_annual_market_return']>0.2]
plot_function(df=temp, xcol='days', ycol='ITM_return', title="Today")
#plot_function(df=updated_df[updated_df['expected_annual_market_return']>0.2], xcol='days', ycol='gap', title="Historical")


In [ ]:
temp.head(2)

In [ ]:
pd.set_option('display.max_rows', None)
final.head()

In [ ]:
final.sort_values(by='ITM_return').head()#(#, ascending=True)


In [ ]:
final.sort_values(by='expected_annual_return',  ascending=False).head()


In [ ]:
select1 = final.days<=5
select2 = final.last_price > .97*final.sma_10
select3 = final.sma_10 > final.sma_50
final[select1 & select2 & select3].sort_values(by='expected_annual_return', ascending=False).head()

In [ ]:
select1 = final.days<=5
select2 = final['P (success)'] > .9
final[select1 & select2].sort_values(by='expected_annual_return', ascending=False).head(10)

In [ ]:
#plot_function(df=all_apreads[all_apreads.short_bid>0], xcol='days', ycol='ITM_return', title="Today")
final.columns

In [ ]:
select1 = final.days<=5
select2 = final.last_price > .97*final.sma_10
select3 = final.sma_10 > final.sma_50
select4 = final.last_price < 20
final[select1 & select2 & select3 & select4].sort_values(by='expected_annual_return', ascending=False).head()